Notebook di elaborazione dei dati delle comunicazioni_sdi per analisi su Tableau

In [1]:
import pandas as pd
import re
from utils.functions import age_clustering, age_sort

Creazione del kpi "Percentuale di donne vittime del seguente reato per classe d'età negli ultimi 5 anni"

il kpi è stato creato nel seguente modo:
- popolazione residente in Italia nel 2022 (anno mediano)
- conteggio delle vittime per tipologia di reato comunicazione_sdi (delitti + codice rosso + reati spia)  e classe di età
- vittime/popolazione residente 2022

In [25]:
df_delitti_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/delitti_vittime.csv")
df_codice_rosso_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/codice_rosso_vittime.csv")
df_reati_spia_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/reati_spia_vittime.csv")
df_popolazione_italia_2022 = pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/resources/popolazione_residente_italia_2022.csv")

In [26]:
"""Mappatura in classi di età della popolazione residente in italia nel 2022"""
df_popolazione_italia_2022 = df_popolazione_italia_2022.drop(df_popolazione_italia_2022.index[-1]) #rimuovo la riga dei totali
df_popolazione_italia_2022["classi_eta"] = df_popolazione_italia_2022["Età"].apply(age_clustering)
df_popolazione_italia_2022_c = df_popolazione_italia_2022[["Totale maschi", "Totale femmine", "classi_eta"]]
df_residenti_classi_eta_2022 = df_popolazione_italia_2022_c.groupby("classi_eta").sum().reset_index()
df_residenti_classi_eta_2022

,classi_eta,Totale maschi,Totale femmine
0,0-13,3555806,3359301
1,14-17,1188841,1114966
2,18-24,2121760,1938579
3,25-34,3182043,3030951
4,35-44,3652999,3627957
5,45-54,4669124,4747983
6,55-64,4285158,4503261
7,65 e oltre,6163225,7888179


In [27]:
"Aggregazioni per classi di età delle vittime nei dataset SDI"
df_delitti_f_classi_eta = df_delitti_vittime.groupby(["eta_alla_data_del_reato_vittima", "delitto"])["valore"].sum().reset_index()
df_delitti_f_classi_eta = df_delitti_f_classi_eta.rename(columns={"delitto": "descrizione_reato"})
df_delitti_f_classi_eta["descrizione_reato"] = df_delitti_f_classi_eta["descrizione_reato"].str.replace(r"^\d+\.\s*", "", regex=True).str.upper()
#print(df_delitti_f_classi_eta)
df_codice_rosso_f_classi_eta = df_codice_rosso_vittime.groupby(["eta_alla_data_del_reato_vittima", "descrizione_reato"])["valore"].sum().reset_index()
#print(df_codice_rosso_f_classi_eta)
df_reati_spia_f_classi_eta = df_reati_spia_vittime.groupby(["eta_alla_data_del_reato_vittima", "descrizione_reato"])["valore"].sum().reset_index()
#print(df_reati_spia_f_classi_eta)
print(df_delitti_f_classi_eta.shape, df_codice_rosso_f_classi_eta.shape, df_reati_spia_f_classi_eta.shape)
df_comunicazioni_sdi = pd.concat([df_delitti_f_classi_eta, df_codice_rosso_f_classi_eta, df_reati_spia_f_classi_eta], axis=0)
print(df_comunicazioni_sdi)

(40, 3) (29, 3) (16, 3)
   eta_alla_data_del_reato_vittima  \
0                             0-13   
1                             0-13   
2                             0-13   
3                             0-13   
4                             0-13   
..                             ...   
11                           45-54   
12                           55-64   
13                           55-64   
14                      65 e oltre   
15                      65 e oltre   

                               descrizione_reato  valore  
0                                        MINACCE    1480  
1                              VIOLENZE SESSUALI    2330  
2                                TENTATI OMICIDI      24  
3                                 LESIONI DOLOSE    2535  
4                                       PERCOSSE     972  
..                                           ...     ...  
11  MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI   23338  
12                              ATTI PERSECUTOR

In [28]:
df_comunicazioni_sdi[df_comunicazioni_sdi["descrizione_reato"] == "COSTRIZIONE O INDUZIONE AL MATRIMONIO"]


,eta_alla_data_del_reato_vittima,descrizione_reato,valore
0,0-13,COSTRIZIONE O INDUZIONE AL MATRIMONIO,3
4,14-17,COSTRIZIONE O INDUZIONE AL MATRIMONIO,29
8,18-24,COSTRIZIONE O INDUZIONE AL MATRIMONIO,48
12,25-34,COSTRIZIONE O INDUZIONE AL MATRIMONIO,11
19,45-54,COSTRIZIONE O INDUZIONE AL MATRIMONIO,2


In [29]:
df_comunicazioni_sdi[df_comunicazioni_sdi["descrizione_reato"] == "COSTRIZIONE O INDUZIONE AL MATRIMONIO"]
nuova_riga = pd.DataFrame([{"eta_alla_data_del_reato_vittima": "35-44", "descrizione_reato": "COSTRIZIONE O INDUZIONE AL MATRIMONIO", "valore": 0}])
nuova_riga_2 = pd.DataFrame([{"eta_alla_data_del_reato_vittima": "55-64", "descrizione_reato": "COSTRIZIONE O INDUZIONE AL MATRIMONIO", "valore": 0}])
nuova_riga_3 = pd.DataFrame([{"eta_alla_data_del_reato_vittima": "65 e oltre", "descrizione_reato": "COSTRIZIONE O INDUZIONE AL MATRIMONIO", "valore": 0}])
df_comunicazioni_sdi = pd.concat([df_comunicazioni_sdi, nuova_riga, nuova_riga_2, nuova_riga_3], axis=0)
df_comunicazioni_sdi[df_comunicazioni_sdi["descrizione_reato"] == "COSTRIZIONE O INDUZIONE AL MATRIMONIO"]


,eta_alla_data_del_reato_vittima,descrizione_reato,valore
0,0-13,COSTRIZIONE O INDUZIONE AL MATRIMONIO,3
4,14-17,COSTRIZIONE O INDUZIONE AL MATRIMONIO,29
8,18-24,COSTRIZIONE O INDUZIONE AL MATRIMONIO,48
12,25-34,COSTRIZIONE O INDUZIONE AL MATRIMONIO,11
19,45-54,COSTRIZIONE O INDUZIONE AL MATRIMONIO,2
0,35-44,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0
0,55-64,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0
0,65 e oltre,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0


In [30]:
"""merge con popolazione residente in italia 2022"""
df_comunicazioni_sdi_f = df_comunicazioni_sdi.merge(df_residenti_classi_eta_2022, how="left", left_on="eta_alla_data_del_reato_vittima", right_on="classi_eta")
df_comunicazioni_sdi_f["p_vittime"]= round(df_comunicazioni_sdi_f["valore"] / (df_comunicazioni_sdi_f["Totale femmine"])*100,2)
df_comunicazioni_sdi_f["ordinamento_classe_eta"] = df_comunicazioni_sdi_f["eta_alla_data_del_reato_vittima"].apply(age_sort)
df_comunicazioni_sdi_f = df_comunicazioni_sdi_f.sort_values(by=["ordinamento_classe_eta", "p_vittime"], ascending=[True, False])
#df_comunicazioni_sdi_f_max=df_comunicazioni_sdi_f.loc[df_comunicazioni_sdi_f.groupby("eta_alla_data_del_reato_vittima")["p_vittime"].idxmax()]
#df_comunicazioni_sdi_f_max

In [31]:
df_totale_vittime_classi_eta = df_comunicazioni_sdi.groupby("eta_alla_data_del_reato_vittima")["valore"].sum()
print(df_totale_vittime_classi_eta) 

eta_alla_data_del_reato_vittima
0-13           13373
14-17          20953
18-24          53984
25-34         105085
35-44         134769
45-54         122829
55-64          63719
65 e oltre     44822
Name: valore, dtype: int64


In [32]:
df_totale_vittime_classi_eta = df_comunicazioni_sdi.merge(df_totale_vittime_classi_eta, how="left", on="eta_alla_data_del_reato_vittima").rename(columns={"valore_y": "totale_vittime_classe_eta", "valore_x": "valore"})
df_totale_vittime_classi_eta

,eta_alla_data_del_reato_vittima,descrizione_reato,valore,totale_vittime_classe_eta
0,0-13,MINACCE,1480,13373
1,0-13,VIOLENZE SESSUALI,2330,13373
2,0-13,TENTATI OMICIDI,24,13373
3,0-13,LESIONI DOLOSE,2535,13373
4,0-13,PERCOSSE,972,13373
...,...,...,...,...
83,65 e oltre,ATTI PERSECUTORI,3748,44822
84,65 e oltre,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,11167,44822
85,35-44,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,134769
86,55-64,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,63719


In [33]:
df_totale_vittime_classi_eta["p_vittime_su_tot_reati"]= round(df_totale_vittime_classi_eta["valore"] / df_totale_vittime_classi_eta["totale_vittime_classe_eta"]*100,2)
df_totale_vittime_classi_eta["ordinamento_classe_eta"] = df_totale_vittime_classi_eta["eta_alla_data_del_reato_vittima"].apply(age_sort)
df_totale_vittime_classi_eta

,eta_alla_data_del_reato_vittima,descrizione_reato,valore,totale_vittime_classe_eta,p_vittime_su_tot_reati,ordinamento_classe_eta
0,0-13,MINACCE,1480,13373,11.07,0
1,0-13,VIOLENZE SESSUALI,2330,13373,17.42,0
2,0-13,TENTATI OMICIDI,24,13373,0.18,0
3,0-13,LESIONI DOLOSE,2535,13373,18.96,0
4,0-13,PERCOSSE,972,13373,7.27,0
...,...,...,...,...,...,...
83,65 e oltre,ATTI PERSECUTORI,3748,44822,8.36,7
84,65 e oltre,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,11167,44822,24.91,7
85,35-44,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,134769,0.00,4
86,55-64,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,63719,0.00,6


In [34]:
df_comunicazioni_sdi_vittime_f = df_totale_vittime_classi_eta.sort_values(by=["ordinamento_classe_eta", "p_vittime_su_tot_reati"], ascending=[True, False])
#df_comunicazioni_sdi_vittime_f_max=df_comunicazioni_sdi_vittime_f.loc[df_comunicazioni_sdi_vittime_f.groupby("eta_alla_data_del_reato_vittima")["p_vittime_su_tot_reati"].idxmax()]
#df_comunicazioni_sdi_vittime_f_max
df_comunicazioni_sdi_vittime_f.to_csv("df_comunicazioni_sdi_vittime_f.csv", index=False)